# Mapping Comparison via H₃⁺

This notebook compares different fermion-to-qubit mappings applied to the H₃⁺ molecule.
Each mapping is evaluated under the same ansatz (UCCSD) and optimizer (Adam).

We analyze:
- Qubit Hamiltonian complexity (Pauli term count),
- Energy convergence over 50 VQE iterations,
- Final ground state energy from each mapping.

Mappings compared:
- Jordan-Wigner (JW)
- Bravyi-Kitaev (BK)
- Parity

In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but AD-compatible
from pennylane import qchem              # Quantum chemistry tools (mappings, HF, excitations)
import matplotlib.pyplot as plt
import os
from vqe_utils import get_optimizer      # Consistent optimizer helper

IMG_DIR = "images"                       # Single image directory used repo-wide
os.makedirs(IMG_DIR, exist_ok=True)

In [ ]:
# Geometry of Trihydrogen Cation (Å; slight distortion to break symmetry)
symbols = ["H", "H", "H"]
coordinates = np.array([
    [ 0.000000,  1.000000,  0.000000],
    [-0.866025, -0.500000,  0.000000],
    [ 0.800000, -0.300000,  0.000000]  # slight distortion
])
electrons = 2

mappings = ["jordan_wigner", "bravyi_kitaev", "parity"]
hamiltonians = {}
qubits_required = {}

for mapping in mappings:
    # Build qubit Hamiltonian for the chosen fermion-to-qubit mapping
    H, qubits = qchem.molecular_hamiltonian(
        symbols, coordinates, charge=+1, mapping=mapping, active_electrons=electrons
    )
    hamiltonians[mapping] = H
    qubits_required[mapping] = qubits
    print(f"{mapping:>15}: {len(H)} Pauli terms with {qubits} qubits")

In [ ]:
# Use Jordan-Wigner mapping qubit count as baseline
qubits = qubits_required["jordan_wigner"]
hf_state = qchem.hf_state(electrons, qubits)
singles, doubles = qchem.excitations(electrons, qubits)
singles = [tuple(s) for s in singles]
doubles = [tuple(d) for d in doubles]


def uccsd_ansatz(params, wires):
    i = 0
    qml.BasisState(hf_state, wires=wires)
    for ex in singles:
        qml.SingleExcitation(params[i], wires=ex)
        i += 1
    for ex in doubles:
        qml.DoubleExcitation(params[i], wires=ex)
        i += 1

max_iter = 50
results = {}

for mapping in mappings:
    dev = qml.device("default.qubit", wires=qubits)
    H = hamiltonians[mapping]

    @qml.qnode(dev)
    def circuit(params):
        uccsd_ansatz(params, wires=range(qubits))
        return qml.expval(H)

    n_params = len(singles) + len(doubles)
    params = np.random.normal(0, 0.1, size=n_params, requires_grad=True)
    opt = get_optimizer("Adam", stepsize=0.2)

    energies = []
    for _ in range(max_iter):
        energies.append(circuit(params))
        params = opt.step(circuit, params)

    results[mapping] = energies

# Plot energy convergence
label_map = {"jordan_wigner": "Jordan–Wigner", "bravyi_kitaev": "Bravyi–Kitaev", "parity": "Parity"}
plt.figure(figsize=(8, 5))
for mapping in mappings:
    plt.plot(results[mapping], label=label_map.get(mapping, mapping))

plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("H3⁺ VQE: Energy Convergence by Mapping (Noiseless, UCCSD)")
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.savefig(f"{IMG_DIR}/H3+_Mapping_Comparison.png", dpi=300)
plt.show()

In [ ]:
# Print final energies
print("\nFinal VQE Energies:")
for mapping in mappings:
    print(f"{mapping:>15}: {results[mapping][-1]:.8f} Ha")